# Single cell data analysis using Seurat

* __Notebook version__: `v0.0.2`
* __Created by:__ `Imperial BRC Genomics Facility`
* __Maintained by:__ `Imperial BRC Genomics Facility`
* __Docker image:__ `imperialgenomicsfacility/seurat-notebook-image:release-v0.0.1`
* __Github repository:__ [imperial-genomics-facility/seurat-notebook-image](https://github.com/imperial-genomics-facility/seurat-notebook-image/)
* __Created on:__ {{ DATE_TAG }}
* __Contact us:__ [Imperial BRC Genomics Facility](https://www.imperial.ac.uk/medicine/research-and-impact/facilities/genomics-facility/contact/)
* __License:__ [Apache License 2.0](https://github.com/imperial-genomics-facility/seurat-notebook-image/blob/master/LICENSE)
* __Project name:__ {{ PROJECT_IGF_ID }}
{% if SAMPLE_IGF_ID %}* __Sample name:__ {{ SAMPLE_IGF_ID }}{% endif %}

This notebook was created using the codes and documentations from the following Seurat tutorial: 

* [Seurat - Guided Clustering Tutorial](https://satijalab.org/seurat/v3.2/pbmc3k_tutorial.html).

## Table of content
* [Load data from Cellranger count output](#Load-data-from-Cellranger-count-output)
* [Standard pre-processing workflow](#Standard-pre-processing-workflow)
  * [QC and selecting cells for further analysis](#QC-and-selecting-cells-for-further-analysis)
* [Normalizing the data](#Normalizing-the-data)
* [Identification of highly variable features (feature selection)](#Identification-of-highly-variable-features-or-feature-selection)
* [Scaling the data](#Scaling-the-data)
* [Perform linear dimensional reduction](#Perform-linear-dimensional-reduction)
* [Determine the dimensionality of the dataset](#Determine-the-dimensionality-of-the-dataset)
* [Cluster the cells](#Cluster-the-cells)
* [Run non-linear dimensional reduction UMAP](#Run-non-linear-dimensional-reduction-UMAP)
* [Finding differentially expressed features or cluster biomarkers](#Finding-differentially-expressed-features-or-cluster-biomarkers)

## Load data from Cellranger count output

In [ ]:
library(dplyr)
library(Seurat)
library(patchwork)
# Load the dataset
cellranger_count_dir = "{{ CELLRANGER_COUNT_DIR }}"
sample_id = "{{ SAMPLE_IGF_ID }}"
sc.data <- Read10X(data.dir = file.path(cellranger_count_dir ,"sample_feature_bc_matrix"))
# Initialize the Seurat object with the raw (non-normalized data).
sc <- CreateSeuratObject(counts = sc.data, project = sample_id, min.cells = 3, min.features = 200)
sc

Data in the count matrix looks like the following

In [ ]:
sc.data[1:5,1:5]

The `.` values in the matrix represent 0s (no molecules detected). Since most values in an scRNA-seq matrix are 0, Seurat uses a sparse-matrix representation whenever possible. This results in significant memory and speed savings for Drop-seq/inDrop/10x data.

<div align="right"><a href="#Table-of-content">Go to TOC</a></div>

## Standard pre-processing workflow

The steps below encompass the standard pre-processing workflow for scRNA-seq data in Seurat. These represent the selection and filtration of cells based on QC metrics, data normalization and scaling, and the detection of highly variable features.

### QC and selecting cells for further analysis

Seurat allows you to easily explore QC metrics and filter cells based on any user-defined criteria. A few QC metrics [commonly used](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4758103/) by the community include

  * The number of unique genes detected in each cell.
    * Low-quality cells or empty droplets will often have very few genes
    * Cell doublets or multiplets may exhibit an aberrantly high gene count
  * Similarly, the total number of molecules detected within a cell (correlates strongly with unique genes)
  * The percentage of reads that map to the mitochondrial genome
    * Low-quality / dying cells often exhibit extensive mitochondrial contamination
    * We calculate mitochondrial QC metrics with the PercentageFeatureSet function, which calculates the percentage of counts originating from a set of features

Here, we are using all the set of genes starting with `MT-` (or `mt-` for mouse) as a set of mitochondrial genes.


In [ ]:
pattern = "^MT-"
genome_build = '{{ GENOME_BUILD }}'

if(genome_build=='HG38'){
    pattern = "^MT-"
}

if(genome_build=='MM10'){
    pattern = "^mt-"
}

sc[["percent.mt"]] <- PercentageFeatureSet(sc, pattern = pattern)

The number of unique genes and total molecules are automatically calculated during `CreateSeuratObject`

In [ ]:
# Show QC metrics for the first 5 cells
head(sc@meta.data, 5)

In the example below, we visualize QC metrics, and use these to filter cells.

* We filter cells that have unique feature counts over 2,500 or less than 200
* We filter cells that have >5% mitochondrial counts


In [ ]:
# Visualize QC metrics as a violin plot
options(repr.plot.width=20, repr.plot.height=8)
VlnPlot(sc, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)

In [ ]:
# FeatureScatter is typically used to visualize feature-feature relationships, but can be used
# for anything calculated by the object, i.e. columns in object metadata, PC scores etc.
options(repr.plot.width=20, repr.plot.height=10)
plot1 <- FeatureScatter(sc, feature1 = "nCount_RNA", feature2 = "percent.mt")
plot2 <- FeatureScatter(sc, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
plot1 + plot2

In [ ]:
sc <- subset(sc, subset = nFeature_RNA > 200 & nFeature_RNA < 2500 & percent.mt < 5)

<div align="right"><a href="#Table-of-content">Go to TOC</a></div>

## Normalizing the data

After removing unwanted cells from the dataset, the next step is to normalize the data. By default, we employ a global-scaling normalization method __LogNormalize__ that normalizes the feature expression measurements for each cell by the total expression, multiplies this by a scale factor (10,000 by default), and log-transforms the result. Normalized values are stored in `sc[["RNA"]]@data`.

In [ ]:
sc <- NormalizeData(sc, normalization.method = "LogNormalize", scale.factor = 10000)

For clarity, in this previous line of code (and in future commands), we provide the default values for certain parameters in the function call. However, this isn't required and the same behavior can be achieved with:

In [ ]:
sc <- NormalizeData(sc)

<div align="right"><a href="#Table-of-content">Go to TOC</a></div>

## Identification of highly variable features or feature selection

We next calculate a subset of features that exhibit high cell-to-cell variation in the dataset (i.e, they are highly expressed in some cells, and lowly expressed in others). Its [reported](https://www.nature.com/articles/nmeth.2645) that focusing on these genes in downstream analysis helps to highlight biological signal in single-cell datasets.

The method implemented in Seurat3 is described in detail [here](https://www.biorxiv.org/content/early/2018/11/02/460147.full.pdf), and improves on previous versions by directly modeling the mean-variance relationship inherent in single-cell data, and is implemented in the FindVariableFeatures function. By default, we return 2,000 features per dataset. These will be used in downstream analysis, like PCA.

In [ ]:
sc <- FindVariableFeatures(sc, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(sc), 10)

# plot variable features with and without labels
plot1 <- VariableFeaturePlot(sc)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
plot1 + plot2

<div align="right"><a href="#Table-of-content">Go to TOC</a></div>

## Scaling the data

Next, we apply a linear transformation ('scaling') that is a standard pre-processing step prior to dimensional reduction techniques like PCA. The `ScaleData` function:

* Shifts the expression of each gene, so that the mean expression across cells is 0
* Scales the expression of each gene, so that the variance across cells is 1
  * This step gives equal weight in downstream analyses, so that highly-expressed genes do not dominate
* The results of this are stored in `pbmc[["RNA"]]@scale.data`

`ScaleData` function can be used for removing unwanted sources of variation from a single-cell dataset. For example, we could 'regress out' heterogeneity associated with (for example) cell cycle stage, or mitochondrial contamination.

In [ ]:
all.genes <- rownames(sc)
sc <- ScaleData(sc, features = all.genes, vars.to.regress = "percent.mt")

<div align="right"><a href="#Table-of-content">Go to TOC</a></div>

## Perform linear dimensional reduction

Next we perform PCA on the scaled data using the previously determined variable features as input.

In [ ]:
sc <- RunPCA(sc, features = VariableFeatures(object = sc))

In [ ]:
# Examine and visualize PCA results a few different ways
print(sc[["pca"]], dims = 1:5, nfeatures = 5)

Visualizing both cells and features that define the PCA using `VizDimReduction`

In [ ]:
options(repr.plot.width=20, repr.plot.height=10)
VizDimLoadings(sc, dims = 1:2, reduction = "pca")

Visualizing both cells and features that define the PCA using `DimPlot`

In [ ]:
options(repr.plot.width=10, repr.plot.height=8)
DimPlot(sc, reduction = "pca")

In particular `DimHeatmap` allows for easy exploration of the primary sources of heterogeneity in a dataset, and can be useful when trying to decide which PCs to include for further downstream analyses. Both cells and features are ordered according to their PCA scores. Setting `cells` to a number plots the 'extreme' cells on both ends of the spectrum, which dramatically speeds plotting for large datasets. Though clearly a supervised analysis, we find this to be a valuable tool for exploring correlated feature sets.

In [ ]:
options(repr.plot.width=14, repr.plot.height=7)
DimHeatmap(sc, dims = 1, cells = 500, balanced = TRUE)

In [ ]:
options(repr.plot.width=12, repr.plot.height=22)
DimHeatmap(sc, dims = 1:15, cells = 500, balanced = TRUE,ncol = 2)

<div align="right"><a href="#Table-of-content">Go to TOC</a></div>

## Determine the dimensionality of the dataset

Here, we are generating an 'Elbow plot': a ranking of principle components based on the percentage of variance explained by each one (`ElbowPlot` function). For this analysis, we are using the first 10 PCs.

In [ ]:
options(repr.plot.width=12, repr.plot.height=7)
ElbowPlot(sc)

<div align="right"><a href="#Table-of-content">Go to TOC</a></div>

## Cluster the cells

Seurat v3 applies a graph-based clustering approach, building upon initial strategies in [Macosko et al](http://www.cell.com/abstract/S0092-8674(15)00549-8). Importantly, the distance metric which drives the clustering analysis (based on previously identified PCs) remains the same. 

Seurat's approach was heavily inspired by recent manuscripts which applied graph-based clustering approaches to scRNA-seq data [SNN-Cliq](http://bioinformatics.oxfordjournals.org/content/early/2015/02/10/bioinformatics.btv088.abstract), [Xu and Su, Bioinformatics, 2015](http://bioinformatics.oxfordjournals.org/content/early/2015/02/10/bioinformatics.btv088.abstract) and CyTOF data [PhenoGraph, Levine et al., Cell, 2015](http://www.ncbi.nlm.nih.gov/pubmed/26095251). Briefly, these methods embed cells in a graph structure - for example a K-nearest neighbor (KNN) graph, with edges drawn between cells with similar feature expression patterns, and then attempt to partition this graph into highly interconnected 'quasi-cliques' or 'communities'.

As in PhenoGraph, it first constructs a KNN graph based on the euclidean distance in PCA space, and refine the edge weights between any two cells based on the shared overlap in their local neighborhoods (Jaccard similarity). This step is performed using the `FindNeighbors` function, and takes as input the previously defined dimensionality of the dataset (first 10 PCs).

To cluster the cells, Seurat uses modularity optimization techniques such as the Louvain algorithm (default) or SLM [SLM, Blondel et al., Journal of Statistical Mechanics](http://dx.doi.org/10.1088/1742-5468/2008/10/P10008), to iteratively group cells together, with the goal of optimizing the standard modularity function. The `FindClusters` function implements this procedure, and contains a resolution parameter that sets the 'granularity' of the downstream clustering, with increased values leading to a greater number of clusters. Its recommended to set this parameter bewteen 0.4-1.2 for single-cell datasets of around 3K cells. Optimal resolution often increases for larger datasets. The clusters can be found using the `Idents` function.

In [ ]:
sc <- FindNeighbors(sc, dims = 1:10)
sc <- FindClusters(sc, resolution = 0.5)

In [ ]:
# Look at cluster IDs of the first 5 cells
head(Idents(sc), 5)

<div align="right"><a href="#Table-of-content">Go to TOC</a></div>

## Run non-linear dimensional reduction UMAP

Seurat offers several non-linear dimensional reduction techniques, such as tSNE and UMAP, to visualize and explore these datasets. The goal of these algorithms is to learn the underlying manifold of the data in order to place similar cells together in low-dimensional space. Cells within the graph-based clusters determined above should co-localize on these dimension reduction plots. As input to the UMAP and tSNE, we suggest using the same PCs as input to the clustering analysis.


In [ ]:
sc <- RunUMAP(sc, dims = 1:10)

In [ ]:
options(repr.plot.width=10, repr.plot.height=8)
DimPlot(sc, reduction = "umap",label=TRUE,label.size=6)

<div align="right"><a href="#Table-of-content">Go to TOC</a></div>

## Finding differentially expressed features or cluster biomarkers

Seurat can help you find markers that define clusters via differential expression. By default, it identifes positive and negative markers of a single cluster (specified in `ident.1`), compared to all other cells. `FindAllMarkers` automates this process for all clusters, but you can also test groups of clusters vs. each other, or against all cells.

The `min.pct` argument requires a feature to be detected at a minimum percentage in either of the two groups of cells, and the thresh.test argument requires a feature to be differentially expressed (on average) by some amount between the two groups. You can set both of these to 0, but with a dramatic increase in time - since this will test a large number of features that are unlikely to be highly discriminatory. As another option to speed up these computations, `max.cells.per.ident` can be set. This will downsample each identity class to have no more cells than whatever this is set to. While there is generally going to be a loss in power, the speed increases can be significiant and the most highly differentially expressed features will likely still rise to the top.

In [ ]:
# find all markers of cluster 1
cluster1.markers <- FindMarkers(sc, ident.1 = 1, min.pct = 0.25)
head(cluster1.markers, n = 5)

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive ones
sc.markers <- FindAllMarkers(sc, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)
sc.markers %>% group_by(cluster) %>% top_n(n = 5, wt = avg_logFC)

Seurat also includes several tools for visualizing marker expression. `DoHeatmap` generates an expression heatmap for given cells and features. In this case, we are plotting the top 10 markers (or all markers if less than 10) for each cluster.

In [ ]:
options(repr.plot.width=20, repr.plot.height=14)
top10 <- sc.markers %>% group_by(cluster) %>% top_n(n = 10, wt = avg_logFC)
DoHeatmap(sc, features = top10$gene) + NoLegend()

<div align="right"><a href="#Table-of-content">Go to TOC</a></div>

## References

* [Seurat - Guided Clustering Tutorial](https://satijalab.org/seurat/v3.2/pbmc3k_tutorial.html)

## Acknowledgement
The Imperial BRC Genomics Facility is supported by NIHR funding to the Imperial Biomedical Research Centre.